# 3nm Nanosheet Figure of Merit Prediction

Created on Oct 30 2020

@author: Haoqing Xu @ICAC, IMECAS

This project focus on 3nm nanosheet FoM prediction with Multiple Layer Perceptron

# Import packages

In [1]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Data Loading

In [20]:
data = np.loadtxt("data.csv",dtype="float",delimiter=',', skiprows=1, usecols=None, unpack=False)

print(data.shape)
print("The size of dataset is", data.shape)

x_train = data[:1400,:3]
y_train = data[:1400, 3:]
x_test = data[1400:,:3]
y_test = data[1400:, 3:]

x_train = StandardScaler().fit_transform(x_train)
y_train = StandardScaler().fit_transform(y_train)
x_test = StandardScaler().fit_transform(x_test)
y_test = StandardScaler().fit_transform(y_test)

x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)

print('Shape and Type of x_train:', x_train.shape, x_train.dtype)
print('Shape and Type of y_train:', y_train.shape, y_train.dtype)
print('Shape and Type of x_test:', x_test.shape, x_test.dtype)
print('Shape and Type of y_test:', y_test.shape, y_test.dtype)

(1968, 7)
The size of dataset is (1968, 7)
Shape and Type of x_train: torch.Size([1400, 3]) torch.float64
Shape and Type of y_train: torch.Size([1400, 4]) torch.float64
Shape and Type of x_test: torch.Size([568, 3]) torch.float64
Shape and Type of y_test: torch.Size([568, 4]) torch.float64


In [3]:
x_train = Variable(x_train)
y_train = Variable(y_train)
x_test = Variable(x_test)
y_test = Variable(y_test)

In [4]:
class Dataset(Dataset):
    def __init__(self,x,y):
        self.x=x
        self.y=y

    def __getitem__(self,index):
        return self.x[index], self.y[index]
  
    def __len__(self):
        return len(self.x)

In [5]:
dataset_train = Dataset(x_train, y_train)
dataset_test = Dataset(x_test, y_test)

print(dataset_train.x.shape, dataset_train.y.shape, dataset_test.x.shape, dataset_test.y.shape)

torch.Size([1400, 3]) torch.Size([1400, 4]) torch.Size([568, 3]) torch.Size([568, 4])


# Neural Network Building

Define the Network Structure

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(3, 25, bias = True) #x.shape=[1968,3], x.shape[1]=3
        self.fc2 = nn.Linear(25, 20, bias = True)
        self.fc3 = nn.Linear(20, 4, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

# Hyper-Parameter and Training Settings 

In [7]:
BATCH_SIZE = 8
EPOCHS = 500
LR = 5e-4

model = MLP()

#optimizer = optim.RMSprop(model.parameters(), lr=LR)
#optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=LR)

In [8]:
import os

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#For multiple GPU usage 
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model, device_ids = [0,1]).cuda()

model = model.to(DEVICE)
print(model)

Let's use 2 GPUs!
DataParallel(
  (module): MLP(
    (fc1): Linear(in_features=3, out_features=25, bias=True)
    (fc2): Linear(in_features=25, out_features=20, bias=True)
    (fc3): Linear(in_features=20, out_features=4, bias=True)
    (sigmoid): Sigmoid()
  )
)


In [9]:
train_loader= torch.utils.data.DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader= torch.utils.data.DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, shuffle=True)

# Training and Test

In [10]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for inputs, labels in train_loader:
        inputs = Variable(inputs.float()).to(DEVICE)
        labels = Variable(labels.float()).to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.mse_loss(outputs, labels)
        
        #loss_func = nn.L1Loss()
        #loss = loss_func(outputs, labels)
        # loss = -(labels * torch.log(output)+ (1-labels) * torch.log(1-output)).mean()
        loss.backward()
        optimizer.step()
    #output = (output>0.5).float()
    #correct = (output == labels).float().sum()
    print("Epoch {}/{}, train_loss:{:.4f}, ".format(epoch+1,EPOCHS,loss.item()), end='')

In [11]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = Variable(inputs.float()).to(DEVICE)
            labels = Variable(labels.float()).to(DEVICE)
            outputs = model(inputs)
            test_loss += F.mse_loss(outputs, labels, reduction='sum').item()
            #pred = torch.tensor([[1] if num[0] >= 0.5 else [0] for num in output]).to(device)
            #correct += (pred == target.long()).sum().item()
        print('test_loss: {:.4f}'.format(test_loss))

In [12]:
for epoch in range(EPOCHS):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Epoch 1/500, train_loss:0.8072, test_loss: 2425.4607
Epoch 2/500, train_loss:0.4299, test_loss: 2325.6183
Epoch 3/500, train_loss:0.4952, test_loss: 2295.6061
Epoch 4/500, train_loss:0.6115, test_loss: 2283.2326
Epoch 5/500, train_loss:1.8204, test_loss: 2276.3481
Epoch 6/500, train_loss:0.3382, test_loss: 2271.0592
Epoch 7/500, train_loss:0.3704, test_loss: 2265.4515
Epoch 8/500, train_loss:1.0081, test_loss: 2257.8342
Epoch 9/500, train_loss:0.4964, test_loss: 2244.1451
Epoch 10/500, train_loss:0.5862, test_loss: 2215.9542
Epoch 11/500, train_loss:0.3074, test_loss: 2159.5697
Epoch 12/500, train_loss:1.3599, test_loss: 2059.5254
Epoch 13/500, train_loss:4.7832, test_loss: 1958.7790
Epoch 14/500, train_loss:0.4444, test_loss: 1900.7926
Epoch 15/500, train_loss:0.2777, test_loss: 1864.7249
Epoch 16/500, train_loss:4.8057, test_loss: 1832.6175
Epoch 17/500, train_loss:0.4116, test_loss: 1812.3148
Epoch 18/500, train_loss:0.5553, test_loss: 1801.8592
Epoch 19/500, train_loss:0.6742, test

Epoch 152/500, train_loss:0.2634, test_loss: 1738.2886
Epoch 153/500, train_loss:0.1872, test_loss: 1736.8804
Epoch 154/500, train_loss:0.3883, test_loss: 1730.3815
Epoch 155/500, train_loss:0.1914, test_loss: 1742.3556
Epoch 156/500, train_loss:0.1399, test_loss: 1721.3690
Epoch 157/500, train_loss:0.7575, test_loss: 1730.1842
Epoch 158/500, train_loss:0.2418, test_loss: 1739.3301
Epoch 159/500, train_loss:0.2381, test_loss: 1725.4715
Epoch 160/500, train_loss:0.4416, test_loss: 1730.8468
Epoch 161/500, train_loss:0.2909, test_loss: 1731.6394
Epoch 162/500, train_loss:0.4305, test_loss: 1738.0320
Epoch 163/500, train_loss:0.2177, test_loss: 1728.6263
Epoch 164/500, train_loss:0.1098, test_loss: 1737.4698
Epoch 165/500, train_loss:0.2922, test_loss: 1715.3620
Epoch 166/500, train_loss:3.0242, test_loss: 1724.1680
Epoch 167/500, train_loss:0.5504, test_loss: 1717.3647
Epoch 168/500, train_loss:0.7781, test_loss: 1724.3100
Epoch 169/500, train_loss:0.3061, test_loss: 1729.8800
Epoch 170/

Epoch 301/500, train_loss:0.9616, test_loss: 1672.5425
Epoch 302/500, train_loss:0.1763, test_loss: 1671.7364
Epoch 303/500, train_loss:0.3807, test_loss: 1677.0238
Epoch 304/500, train_loss:0.4036, test_loss: 1670.2292
Epoch 305/500, train_loss:0.1643, test_loss: 1667.7730
Epoch 306/500, train_loss:0.2656, test_loss: 1671.0191
Epoch 307/500, train_loss:0.4311, test_loss: 1675.7628
Epoch 308/500, train_loss:0.2368, test_loss: 1665.2315
Epoch 309/500, train_loss:1.5219, test_loss: 1667.9550
Epoch 310/500, train_loss:0.3761, test_loss: 1663.1323
Epoch 311/500, train_loss:0.2538, test_loss: 1662.1808
Epoch 312/500, train_loss:4.2880, test_loss: 1665.6123
Epoch 313/500, train_loss:0.1907, test_loss: 1666.4944
Epoch 314/500, train_loss:0.2000, test_loss: 1661.7196
Epoch 315/500, train_loss:0.2908, test_loss: 1663.7906
Epoch 316/500, train_loss:0.0669, test_loss: 1667.0737
Epoch 317/500, train_loss:0.2411, test_loss: 1658.5213
Epoch 318/500, train_loss:0.7747, test_loss: 1667.7090
Epoch 319/

Epoch 450/500, train_loss:0.8843, test_loss: 1642.0310
Epoch 451/500, train_loss:0.8119, test_loss: 1638.1401
Epoch 452/500, train_loss:0.1334, test_loss: 1634.6419
Epoch 453/500, train_loss:0.8189, test_loss: 1630.4835
Epoch 454/500, train_loss:0.3456, test_loss: 1638.4708
Epoch 455/500, train_loss:0.1954, test_loss: 1638.5096
Epoch 456/500, train_loss:1.0676, test_loss: 1633.4070
Epoch 457/500, train_loss:4.2853, test_loss: 1639.6017
Epoch 458/500, train_loss:1.5860, test_loss: 1634.0896
Epoch 459/500, train_loss:0.5206, test_loss: 1635.9989
Epoch 460/500, train_loss:0.2829, test_loss: 1632.5872
Epoch 461/500, train_loss:0.1796, test_loss: 1634.2239
Epoch 462/500, train_loss:4.8751, test_loss: 1637.5766
Epoch 463/500, train_loss:1.3035, test_loss: 1635.4206
Epoch 464/500, train_loss:3.8045, test_loss: 1638.7720
Epoch 465/500, train_loss:0.2604, test_loss: 1632.0863
Epoch 466/500, train_loss:0.2389, test_loss: 1634.0705
Epoch 467/500, train_loss:0.5520, test_loss: 1635.8168
Epoch 468/

The below are codes for checking GPUs.

In [13]:
torch.cuda.is_available()

True

In [14]:
torch.cuda.device_count()

2

In [15]:
torch.cuda.current_device()

0

In [16]:
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-16GB'

In [17]:
torch.cuda.get_device_name(1)

'Tesla V100-PCIE-16GB'